## 初始化安装包和文件目录

In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

## 输入数据目录
data_dir = "cureus"
inputdirectory = Path(f"./data_input/{data_dir}")
## 输出的 csv 文件保存目录
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

## 加载文件

In [2]:
## Dir PDF Loader
# loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
# loader = PyPDFLoader("./data/MedicalDocuments/orf-path_health-n1.pdf")
loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("数据块数量 = ", len(pages))
# print(pages[3].page_content)


100%|██████████| 1/1 [00:03<00:00,  3.14s/it]

数据块数量 =  3


## 创建一个包含所有数据块的数据集

In [3]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(3, 3)


,text,source,chunk_id
0,以下是雷达研发各细分领域下具体技术或课题的研究报告样例，涵盖系统架构、信号处理、射频硬件、天...,data_input\cureus\20230719-yd-yuechuan3.txt,dfd37b9d8feb4f9aa7dbdd3d4e05365f
1,研究背景：传统雷达与通信系统独立设计，导致频谱资源紧张和设备冗余。本研究探索利用智能超表面实...,data_input\cureus\20230719-yd-yuechuan3.txt,0825caf546544a028aac41c9caebac4a
2,数据分析：给出探测距离、跟踪精度、虚警率等关键性能指标的统计结果，并与研制总要求进行符合性判...,data_input\cureus\20230719-yd-yuechuan3.txt,861066de447c4c27a2e52d11fe7bef50


## 从文本中提取信息

In [4]:
## 该函数使用 helpers/prompt 函数从文本中提取关键信息
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

如果 regenerate 设置为 True，则重新生成数据集，并且两个数据集都以 csv 格式写入。

        dfne = dataframe of edges

        df = dataframe of chunks


否则从输出目录读取数据集


**备注说明**：如设置为true，需安装对应的模型
模型列表地址：https://ollama.ai/library

本项目使用 zephyr 模型，

模型安装 CLI：

```shell
ollama run zephyr "Why is the sky blue?"
```

In [5]:
## 要使用 LLM 重新生成图形，请将此设置为 True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model='qwen/qwen3-vl-4b')
    # concepts_list = df2Graph(df, model='mistral-7b-evol-instruct-chinese:latest')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## 将关系的权重增至 4。
## 稍后计算上下文接近度时，我们将把权重设为 1。 
print(dfg1.shape)
dfg1.head()



ERROR ### Here is the buggy response:  [
  {
    "node_1": "数据分析",
    "node_2": "探测距离",
    "edge": "数据分析用于统计探测距离等关键性能指标"
  },
  {
    "node_1": "数据分析",
    "node_2": "跟踪精度",
    "edge": "数据分析用于统计跟踪精度等关键性能指标"
  },
  {
    "node_1": "数据分析",
    "node_2": "虚警率",
    "edge": "数据分析用于统计虚警率等关键性能指标"
  },
  {
    "node_1": "数据分析",
    "node_2": "研制总要求",
    "edge": "数据分析用于与研制总要求进行符合性判定"
  },
  {
    "node_1": "结论",
    "node_2": "系统各项功能正常",
    "edge": "结论指出系统各项功能运行正常"
  },
  {
    "node_1": "结论",
    "node_2": "性能指标达标",
    "edge": "结论指出系统性能指标达标"
  },
  {
    "node_1": "结论",
    "node_2": "短期失跟现象",
    "edge": "结论指出在特定密集干扰模式下存在短期失跟现象"
  },
  {
    "node_1": "结论",
    "node_2": "算法升级建议",
    "edge": "结论提出针对短期失跟现象的算法升级建议"
  },
  {
    "node_1": "报告样例2",
    "node_2": "毫米波雷达PCB板级信号完整性仿真与测试研究",
    "edge": "报告样例2的标题为《毫米波雷达PCB板级信号完整性仿真与测试研究》"
  },
  {
    "node_1": "研究背景",
    "node_2": "高频信号传输失真",
    "edge": "研究背景指出高频信号易产生失真影响测距测速精度"
  },
  {
    "node_1": "研究背景",
    "node_2": "雷达测距测速精度",
  

,node_1,node_2,edge,chunk_id,count
0,全极化有源相控阵雷达系统架构,气象探测,全极化有源相控阵雷达系统架构针对气象探测领域对高精度、多维度观测的需求提出，以提升目标分类能力。,dfd37b9d8feb4f9aa7dbdd3d4e05365f,4
1,全极化有源相控阵雷达系统架构,粤港澳大湾区天气观测网,该架构在粤港澳大湾区建成国内首个超精细化天气观测网，覆盖华南地区气象局及科研院所。,dfd37b9d8feb4f9aa7dbdd3d4e05365f,4
2,fmcw雷达信号处理算法,多目标场景距离与速度测量,FMCW雷达信号处理算法优化旨在提升在多目标场景下的距离与速度测量精度。,dfd37b9d8feb4f9aa7dbdd3d4e05365f,4
3,fmcw雷达信号处理算法,二次fft处理,采用二次FFT处理多帧数据，生成距离-多普勒图，提取目标距离与速度信息。,dfd37b9d8feb4f9aa7dbdd3d4e05365f,4
4,fmcw雷达信号处理算法,hamming窗函数,引入Hamming窗函数平衡主峰明显度与频谱泄露，优化目标分离效果。,dfd37b9d8feb4f9aa7dbdd3d4e05365f,4


## 计算上下文接近度

In [6]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## 将数据集合转换成节点列表
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # 以块id 为关键字的自连接 在同一文本块之间创建链接。
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # 减少自循环
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## 对边缘进行分组和计数。
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # 边缘掉落 1 次
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
2759,面向6g的智能超表面赋能的通信感知一体化雷达架构研究,全极化有源相控阵雷达系统架构,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",2,contextual proximity
2767,面向6g的智能超表面赋能的通信感知一体化雷达架构研究,智能超表面赋能的通信感知一体化雷达架构,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",2,contextual proximity
2769,面向6g的智能超表面赋能的通信感知一体化雷达架构研究,汽车毫米波雷达天线,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",3,contextual proximity
2776,面向6g的智能超表面赋能的通信感知一体化雷达架构研究,雷达研发细分领域,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",11,contextual proximity
2777,面向6g的智能超表面赋能的通信感知一体化雷达架构研究,雷达系统外场试验方法,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",3,contextual proximity


### 合并数据集

In [7]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,360度无死角覆盖,doherty功率放大器,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",contextual proximity,2
1,360度无死角覆盖,fmcw雷达信号处理算法,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",contextual proximity,3
2,360度无死角覆盖,全极化有源相控阵雷达系统架构,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",contextual proximity,2
3,360度无死角覆盖,智能超表面赋能的通信感知一体化雷达架构,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",contextual proximity,2
4,360度无死角覆盖,汽车毫米波雷达天线,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",contextual proximity,3
...,...,...,...,...,...
395,面向6g的智能超表面赋能的通信感知一体化雷达架构研究,智能超表面赋能的通信感知一体化雷达架构,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",contextual proximity,2
396,面向6g的智能超表面赋能的通信感知一体化雷达架构研究,汽车毫米波雷达天线,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",contextual proximity,3
397,面向6g的智能超表面赋能的通信感知一体化雷达架构研究,雷达研发细分领域,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",contextual proximity,11
398,面向6g的智能超表面赋能的通信感知一体化雷达架构研究,雷达系统外场试验方法,"dfd37b9d8feb4f9aa7dbdd3d4e05365f,dfd37b9d8feb4...",contextual proximity,3


## 计算 NetworkX 图形

In [8]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(74,)

In [9]:
import networkx as nx
G = nx.Graph()

## 为图表添加节点
for node in nodes:
    G.add_node(
        str(node)
    )

## 为图形添加边界
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

### 计算共同体以对节点进行着色

In [10]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("共同体数量 = ", len(communities))
print(communities)

共同体数量 =  24
[['10,000个样本', '实测hrrp数据'], ['360度无死角覆盖'], ['64通道相控阵天线', 'doherty功率放大器', 'doherty功率放大器设计', 'fmcw雷达信号处理算法', 'fmcw雷达信号处理算法优化', 'hamming窗函数', '二次fft处理', '信号处理领域', '全息mimo雷达架构', '全极化有源相控阵雷达系统架构', '功率效率提升40%', '动态目标测试', '多目标场景距离与速度测量', '天线技术领域', '射频硬件领域', '抗干扰测试', '智能反射面（isr）技术', '智能超表面赋能的通信感知一体化雷达架构', '气象探测', '汽车毫米波雷达天线', '汽车毫米波雷达天线优化', '测试验证领域', '粤港澳大湾区天气观测网', '系统架构领域', '通信感知一体化', '雷达射频前端', '雷达研发细分领域', '雷达系统外场试验方法', '雷达系统架构创新研究', '面向6g的智能超表面赋能的通信感知一体化雷达架构研究'], ['77ghz车载雷达', 'sige bicmos单片集成电路'], ['dbscan聚类算法', '联合概率数据关联滤波器'], ['gan工艺', 't/r组件设计'], ['pcb板级信号完整性仿真', '毫米波雷达'], ['一维卷积神经网络', '注意力机制'], ['传统识别方法稳健性不足', '高分辨率距离像'], ['传统雷达与通信系统', '独立设计'], ['信号处理板卡设计', '商用fpga'], ['信噪比10db', '平均识别率95.2%'], ['减少t/r组件数量', '稀疏阵列天线技术'], ['发射通道输出功率≥12dbm', '工作频率76-81ghz'], ['噪声系数≤8db', '接收通道'], ['城市复杂交通场景', '目标跟踪连续性提升25%'], ['基站与雷达信号协同处理', '超表面编码优化算法'], ['外场动态性能测试', '有源相控阵雷达'], ['强风、高湿度环境', '防护等级设计'], ['微通道液冷散热', '结温控制'], ['探测实验', '电子干扰手段'], ['智能超表面'], ['机械扫描气象雷达', '船只晃动环境数据失真'], ['系统功耗', '降

### 创建共同体颜色数据框

In [11]:
import seaborn as sns
palette = "hls"

## 现在将这些颜色添加到共同体中，并制作另一个数据集
def colors2Community(communities) -> pd.DataFrame:
    ## 定义调色板
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,"10,000个样本",#db5770,1
1,实测hrrp数据,#db5770,1
2,360度无死角覆盖,#db5f57,2
3,64通道相控阵天线,#db8057,3
4,doherty功率放大器,#db8057,3
...,...,...,...
69,智能超表面,#db57d3,22
70,机械扫描气象雷达,#a157db,23
71,船只晃动环境数据失真,#a157db,23
72,系统功耗,#dba157,24


### 为图表添加颜色

In [12]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [13]:
from pyvis.network import Network

graph_output_directory = "./docs/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

./docs/index.html
